## Load Dataset

source: https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators

In [2]:
pip install ucimlrepo


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [54]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, RepeatedStratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

In [69]:
def load_dataset():
    # Load the dataset
    cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 

    # Define categorical and numerical features
    categorical_features = ['HighBP', 'HighChol','CholCheck','Smoker','Stroke','HeartDiseaseorAttack',
                            'PhysActivity','Fruits','Veggies','HvyAlcoholConsump','AnyHealthcare','NoDocbcCost','GenHlth',
                            'DiffWalk','Sex','Age','Education','Income']
    numerical_features = ['BMI','MentHlth','PhysHlth',]

    # Get features and target variable
    X = cdc_diabetes_health_indicators.data.features
    y = cdc_diabetes_health_indicators.data.targets['Diabetes_binary'] 

    # Preprocessing: One-hot encoding for categorical variables and scaling for numerical variables
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    # Split the dataset into training and testing sets with a fixed random state for reproducibility
    X_train_full, X_test_full, y_train, y_test, gender_train, gender_test = train_test_split(X, y, 
                                                                                             cdc_diabetes_health_indicators.data.features['Sex'],
                                                                                             test_size=0.2, random_state=42)
    # Apply preprocessing to training and testing set separately
    X_train_processed = preprocessor.fit_transform(X_train_full)
    X_test_processed = preprocessor.transform(X_test_full)

    # Return processed training and testing sets along with gender attributes
    return X_train_processed, X_test_processed, y_train, y_test, gender_train.values, gender_test.values

In [ ]:
# cdc_diabetes_health_indicators.metadata
# cdc_diabetes_health_indicators.variables

## Train Model and Generate Predictions

In [70]:
def train_and_predict_model(X_train, X_test, y_train, weights=None):

    # Initialize the Logistic Regression model
    model = LogisticRegression(max_iter=10000, random_state=0)

    # Train the Logistic Regression model
    model.fit(X_train, y_train, sample_weight=weights)

    # Predict on the testing set
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    return y_pred, y_pred_proba

In [71]:
def determine_confusion_matrix(df):
    if df['y_true'] == df['y_pred'] == 1:
        return 'TP'
    elif df['y_pred'] == 1 and df['y_true'] != df['y_pred']:
        return 'FP'
    elif df['y_true'] == df['y_pred'] == 0:
        return 'TN'
    else:
        return 'FN'

In [91]:
# preprocess and load the data
X_train, X_test, y_train, y_test, gender_train, gender_test = load_dataset()

# train a model and obtain predictions on the test set
y_pred, y_pred_proba = train_and_predict_model(X_train, X_test, y_train)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print('Accuracy:', accuracy, '\nAUC:', auc)

Accuracy: 0.8676285083569851 
AUC: 0.8288682104560867


In [89]:
# try neural net
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(21,), random_state=1)
clf.fit(X_train, y_train)

y_pred1 = clf.predict(X_test)
y_pred_proba1 = clf.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print('Accuracy:', accuracy, '\nAUC:', auc)

/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(21,), random_state=1,
              solver='lbfgs')

In [ ]:
# try svm
from sklearn import svm

clf2 = svm.SVC()
clf2.fit(X_train, y_train)

y_pred2 = clf2.predict(X_test)
y_pred_proba2 = clf2.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print('Accuracy:', accuracy, '\nAUC:', auc)

In [74]:
# Female = 1 and Male = 0
fair_df = pd.DataFrame({'sex': gender_test, 'y_true': y_test, 'y_pred': y_pred})
fair_df['confusion_matrix'] = fair_df[['y_true','y_pred']].apply(determine_confusion_matrix, axis=1)
fair_df.head()

,sex,y_true,y_pred,confusion_matrix
219620,0,0,0,TN
132821,0,0,0,TN
151862,1,0,0,TN
139717,1,0,0,TN
239235,0,0,0,TN


In [ ]:
# 1 stand for female and 0 is male
fair_df['sex'].value_counts()

## Fairness Evaluation

In [76]:
threshold = 0.001

In [77]:
def statistical_parity(df):
    """
    TODO: Add your code here
    """
    female_positive = df[(df['sex'] == 1) & (df['y_pred'] == 1)].shape[0]
    female_positive_prob = female_positive / (df[(df['sex'] == 1)].shape[0])
    
    male_positive = df[(df['sex'] == 0) & (df['y_pred'] == 1)].shape[0]
    male_positive_prob = male_positive / (df[(df['sex'] == 0)].shape[0])

    print('Female Probability of Positive Predictions: %.3f' % female_positive_prob)
    print('Male Probability of Positive Predictions: %.3f' % male_positive_prob)
    
    abs_difference = abs(female_positive_prob - male_positive_prob)
    print('Achieves Statistical Parity: %r' % (abs_difference < threshold))

In [78]:
def predictive_parity(df):
    """
    TODO: Add your code here
    """
    female_TP = df[(df['sex'] == 1) & (df['confusion_matrix'] == 'TP')].shape[0]
    PPV_female = female_TP / (df[(df['sex'] == 1) & (df['y_pred'] == 1)].shape[0])
    
    male_TP = df[(df['sex'] == 0) & (df['confusion_matrix'] == 'TP')].shape[0]
    PPV_male = male_TP / (df[(df['sex'] == 0) & (df['y_pred'] == 1)].shape[0])

    print('Female Probability of True Positive Predictions: %.3f' % PPV_female)
    print('Male Probability of True Positive Predictions: %.3f' % PPV_male)
    
    abs_difference = abs(PPV_female - PPV_male)
    print('Achieves Statistical Parity: %r' % (abs_difference < threshold))

In [79]:
def equalized_odds(df):
    """
    TODO: Add your code here
    """
    # FNR = FN/(FN+TP) = FN/(all-positive-true-label)
    female_fn = df[(df['sex'] == 1) & (df['confusion_matrix'] == 'FN')].shape[0]
    fnr_female = female_fn / (df[(df['sex'] == 1) & (df['y_true'] == 1)].shape[0])
    male_fn = df[(df['sex'] == 0) & (df['confusion_matrix'] == 'FN')].shape[0]
    fnr_male = male_fn / (df[(df['sex'] == 0) & (df['y_true'] == 1)].shape[0])
    
    # FPR = FP/(FP+TN) = FN/(all-negative-true-label)
    female_fp = df[(df['sex'] == 1) & (df['confusion_matrix'] == 'FP')].shape[0]
    fpr_female = female_fp / (df[(df['sex'] == 1) & (df['y_true'] == 0)].shape[0])
    male_fp = df[(df['sex'] == 0) & (df['confusion_matrix'] == 'FP')].shape[0]
    fpr_male = male_fp / (df[(df['sex'] == 0) & (df['y_true'] == 0)].shape[0])

    print('Probability of Credit-Worthy Female Predicted Not Credit-Worthy: %.3f' % fnr_female)
    print('Probability of Credit-Worthy Male Predicted Not Credit-Worthy: %.3f' % fnr_male)
    
    abs_difference_fnr = abs(fnr_female - fnr_male)
    print('Achieves Equality of Non Credit Worthy Prediction: %r' % (abs_difference_fnr < threshold))
    
    print('Probability of Non Credit-Worthy Female Predicted Credit-Worthy: %.3f' % fpr_female)
    print('Probability of Non Credit-Worthy Male Predicted Credit-Worthy: %.3f' % fpr_male)
    
    abs_difference_fpr = abs(fnr_female - fnr_male)
    print('Achieves Equality of Credit Worthy Prediction: %r' % (abs_difference_fpr < threshold))

In [80]:
def accuracy_equality(df):
    """
    TODO: Add your code here
    """

    # Accuracy = (TP+TN)/all-samples
    female_t = df[(df['sex'] == 1) & (df['confusion_matrix'].isin(['TP', 'TN']))].shape[0]
    accuracy_female = female_t / (df[(df['sex'] == 1)].shape[0])
    male_t = df[(df['sex'] == 0) & (df['confusion_matrix'].isin(['TP', 'TN']))].shape[0]
    accuracy_male = male_t / (df[(df['sex'] == 0)].shape[0])
    
    print('Female Accuracy: %.3f' % accuracy_female)
    print('Male Accuracy: %.3f' % accuracy_male)
    
    abs_difference = abs(accuracy_female - accuracy_male)
    print('Equality of Accuracy: %r' % (abs_difference < threshold))

In [81]:
def treatment_equality(df):
    """
    TODO: Add your code here
    """
    
    female = df[(df['sex'] == 1)]
    ratio_female = (female[female['confusion_matrix'] == 'FN'].shape[0] / 
        female[female['confusion_matrix'] == 'FP'].shape[0])

    male = df[(df['sex'] == 0)]
    ratio_male = (male[male['confusion_matrix'] == 'FN'].shape[0] / 
        male[male['confusion_matrix'] == 'FP'].shape[0])

    print('Female Ratio of Errors: %.3f' % ratio_female)
    print('Male Ratio of Errors: %.3f' % ratio_male)
    
    abs_difference = abs(ratio_female - ratio_male)
    print('Achieves Treatment Equality: %r' % (abs_difference < threshold))

In [82]:
statistical_parity(fair_df)
predictive_parity(fair_df)
equalized_odds(fair_df)
accuracy_equality(fair_df)
treatment_equality(fair_df)

Female Probability of Positive Predictions: 0.043
Male Probability of Positive Predictions: 0.036
Achieves Statistical Parity: False
Female Probability of True Positive Predictions: 0.572
Male Probability of True Positive Predictions: 0.570
Achieves Statistical Parity: False
Probability of Credit-Worthy Female Predicted Not Credit-Worthy: 0.835
Probability of Credit-Worthy Male Predicted Not Credit-Worthy: 0.842
Achieves Equality of Non Credit Worthy Prediction: False
Probability of Non Credit-Worthy Female Predicted Credit-Worthy: 0.022
Probability of Non Credit-Worthy Male Predicted Credit-Worthy: 0.018
Achieves Equality of Credit Worthy Prediction: False
Female Accuracy: 0.857
Male Accuracy: 0.876
Equality of Accuracy: False
Female Ratio of Errors: 6.795
Male Ratio of Errors: 7.092
Achieves Treatment Equality: False


## Mitigation through Post-Processiong